In [5]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

# Define the structure of the Alarm Bayesian Network
model = BayesianNetwork([('Burglary', 'Alarm'),
                         ('Earthquake', 'Alarm'),
                         ('Alarm', 'JohnCalls'),
                         ('Alarm', 'MaryCalls')])




In [18]:
result1 = model.is_dconnected('Earthquake', 'Burglary', observed=['Alarm'])
print("Earthquake is independent of Burglary given Alarm:", not result1)


result2 = model.is_dconnected('Earthquake', 'Burglary', observed=[])
print("Earthquake is independent of Burglary:", not result2)


result3 = model.is_dconnected('JohnCalls', 'MaryCalls', observed=['Alarm'])
print("JohnCalls is independent of MaryCalls given Alarm:",not result3)


result4 = model.is_dconnected('JohnCalls', 'MaryCalls', observed=[])
print("JohnCalls is independent of MaryCalls", not result4)

Earthquake is independent of Burglary given Alarm: False
Earthquake is independent of Burglary: True
JohnCalls is independent of MaryCalls given Alarm: True
JohnCalls is independent of MaryCalls False


In [19]:
independencies = model.get_independencies()
print(independencies)

(Burglary ⟂ Earthquake)
(Burglary ⟂ MaryCalls, JohnCalls | Alarm)
(Burglary ⟂ JohnCalls | MaryCalls, Alarm)
(Burglary ⟂ MaryCalls | JohnCalls, Alarm)
(Burglary ⟂ MaryCalls, JohnCalls | Earthquake, Alarm)
(Burglary ⟂ JohnCalls | MaryCalls, Earthquake, Alarm)
(Burglary ⟂ MaryCalls | JohnCalls, Earthquake, Alarm)
(MaryCalls ⟂ Burglary, JohnCalls, Earthquake | Alarm)
(MaryCalls ⟂ JohnCalls, Earthquake | Burglary, Alarm)
(MaryCalls ⟂ Burglary, Earthquake | JohnCalls, Alarm)
(MaryCalls ⟂ Burglary, JohnCalls | Earthquake, Alarm)
(MaryCalls ⟂ Earthquake | Burglary, JohnCalls, Alarm)
(MaryCalls ⟂ JohnCalls | Burglary, Earthquake, Alarm)
(MaryCalls ⟂ Burglary | JohnCalls, Earthquake, Alarm)
(JohnCalls ⟂ Burglary, Earthquake, MaryCalls | Alarm)
(JohnCalls ⟂ MaryCalls, Earthquake | Burglary, Alarm)
(JohnCalls ⟂ Burglary, MaryCalls | Earthquake, Alarm)
(JohnCalls ⟂ Burglary, Earthquake | MaryCalls, Alarm)
(JohnCalls ⟂ MaryCalls | Burglary, Earthquake, Alarm)
(JohnCalls ⟂ Earthquake | Burglary, Mary

In [20]:
# Define the CPDs (Conditional Probability Distributions)

# Burglary has no parent nodes
cpd_burglary = TabularCPD(variable='Burglary', variable_card=2,
                          values=[[0.999], [0.001]])

# Earthquake has no parent nodes
cpd_earthquake = TabularCPD(variable='Earthquake', variable_card=2,
                            values=[[0.998], [0.002]])

# Alarm depends on Burglary and Earthquake
cpd_alarm = TabularCPD(variable='Alarm', variable_card=2,
                       values=[[0.999, 0.71, 0.06, 0.05],  # P(Alarm=0 | B, E)
                               [0.001, 0.29, 0.94, 0.95]],  # P(Alarm=1 | B, E)
                       evidence=['Burglary', 'Earthquake'],
                       evidence_card=[2, 2])

# JohnCalls depends on Alarm
cpd_john_calls = TabularCPD(variable='JohnCalls', variable_card=2,
                            values=[[0.95, 0.1],  # P(JohnCalls=0 | Alarm)
                                    [0.05, 0.9]],  # P(JohnCalls=1 | Alarm)
                            evidence=['Alarm'],
                            evidence_card=[2])

# MaryCalls depends on Alarm
cpd_mary_calls = TabularCPD(variable='MaryCalls', variable_card=2,
                            values=[[0.99, 0.3],  # P(MaryCalls=0 | Alarm)
                                    [0.01, 0.7]],  # P(MaryCalls=1 | Alarm)
                            evidence=['Alarm'],
                            evidence_card=[2])

In [21]:
# Add the CPDs to the model
model.add_cpds(cpd_burglary, cpd_earthquake, cpd_alarm, cpd_john_calls, cpd_mary_calls)

# Verify if the model is valid
assert model.check_model()

# Print the CPDs to verify
print("Conditional Probability Distributions (CPDs):")
for cpd in model.get_cpds():
    print(cpd)

Conditional Probability Distributions (CPDs):
+-------------+-------+
| Burglary(0) | 0.999 |
+-------------+-------+
| Burglary(1) | 0.001 |
+-------------+-------+
+---------------+-------+
| Earthquake(0) | 0.998 |
+---------------+-------+
| Earthquake(1) | 0.002 |
+---------------+-------+
+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.999         | 0.71          | 0.06          | 0.05          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.001         | 0.29          | 0.94          | 0.95          |
+------------+---------------+---------------+---------------+----------

In [25]:
# Perform inference on the model
inference = VariableElimination(model)

# Example Query: Probability of a Burglary given that both John and Mary called
query_result = inference.query(variables=['Burglary'], evidence={'JohnCalls': 1, 'MaryCalls': 1})
print("\nProbability of Burglary given that both John and Mary called:")
print(query_result)


Probability of Burglary given that both John and Mary called:
+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7158 |
+-------------+-----------------+
| Burglary(1) |          0.2842 |
+-------------+-----------------+
